In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import jax.numpy as jnp
import matplotlib.pyplot as plt
import os
from PacTimeOrig.data import DataProcessing as dp
from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.controllers import simulator as sim
from PacTimeOrig.controllers import JaxMod as jm
from PacTimeOrig.controllers import models as conmod
from PacTimeOrig.controllers import utils as ut



In [ ]:

folder='/Users/user/PycharmProjects/PacManMain/data/HumanEMU'
subjects=os.listdir(folder)

#Dataloader, + #sessvar maker,
sessvars, neural=dh.dataloader_EMU(folder=folder,subj=subjects[3])

dataall=neural['neuronData']

# position getter and scaler (all alignedf already to chase_start)
positions=dh.retrievepositions(dataall,dattype='hemu')

#compute derivatives
kinematics=dp.computederivatives(positions, vartodiff=['selfXpos','selfYpos','prey1Xpos','prey1Ypos','prey2Xpos','prey2Ypos'], dt=1.0/60.0,smooth=True)

#get psth
psth=dh.get_psth_EMU(dataall)


#subselect 2 prey trials
ogsessvars=sessvars
kinematics,sessvars =dh.subselect(kinematics,sessvars,trialtype='2')
psth,_= dh.subselect(psth,ogsessvars,trialtype='2')


# Rt compute and cut data
sessvars = dp.get_reaction_time(sessvars, kinematics)
kinematics = dh.cut_to_rt(kinematics, sessvars)
psth = dh.cut_to_rt(psth, sessvars)
kinematics = dh.get_time_vector(kinematics)
kinematics = dp.compute_distance(kinematics,trialtype=2)
#compute relative normalized speed
kinematics = dp.compute_relspeed(kinematics,trialtype=2)
kinematics = dp.compute_selfspeed(kinematics)


#For each kinematics frame, add relative reward value
Xdsgn = kinematics
for trial in range(len(Xdsgn)):
    Xdsgn[trial]['val1'] = np.repeat(sessvars.iloc[trial].NPCvalA,len(kinematics[trial]))
    Xdsgn[trial]['val2'] = np.repeat(sessvars.iloc[trial].NPCvalB,len(kinematics[trial]))
    
#Switch reward positions so highest value is always in prey 1 slot
Xdsgn=dh.rewardalign(Xdsgn)
Xdsgn = [df[sorted(df.columns)] for df in Xdsgn]

#Compute relative value
Xdsgn = [df.assign(relvalue=df['val1']-df['val2']).round(2) for df in Xdsgn]

speed=[]
for trial in range(len(Xdsgn)):
    speed.append(np.array(Xdsgn[trial]['selfspeedmag']))
    
sns.histplot(np.concatenate(speed))

Test model fit

In [ ]:
trial=1
fitdat=ut.get_data_for_fit(Xdsgn, trial)
xin=fitdat['player_pos']
# xin=np.hstack((fitdat['player_pos'],fitdat['player_vel']))

In [ ]:
num_rbfs = 50

A,B=ut.define_system_parameters(ctrltype='p')

#Make time
tmp=ut.make_timeline(fitdat)

#pre inputs
inputs=ut.prepare_inputs(A, B, xin, fitdat['uout'], fitdat['pry1_pos'], fitdat['pry2_pos'], tmp, num_rbfs, xin[:,2:], fitdat['pry1_vel'], fitdat['pry2_vel'], ctrltype='p',usingJax=True)

params = jm.initialize_parameters_p(inputs)

loss_function = jm.create_loss_function_p(ut.generate_rbf_basis,inputs['num_rbfs'])

grad_loss = jm.compute_loss_gradient(loss_function)



# Set up the optimizer
optimizer, opt_state = jm.setup_optimizer(params, learning_rate=1e-2)

# Number of optimization steps
num_steps = 20000
# Optimization loop
for step in range(num_steps):
    params, opt_state, loss = jm.optimization_step(params, opt_state, optimizer, loss_function, inputs)

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")
        
        
#transform paramteres to correct domain
Ls_fit=jnp.log(1+jnp.exp(params[1][1:].reshape(2,1)))

weights = params[0]
widths = params[1][0]
w1,w2 = ut.generate_sim_switch(inputs, widths, weights)
        

In [ ]:
trial=3
num_rbfs = 30


fitdat=ut.get_data_for_fit(Xdsgn, trial)

#Make time
tmp=ut.make_timeline(fitdat)

xin=np.hstack((fitdat['player_pos'],fitdat['player_vel']))
A,B=ut.define_system_parameters(ctrltype='pv')

inputs=ut.prepare_inputs(A, B, xin, fitdat['uout'], fitdat['pry1_pos'], fitdat['pry2_pos'], tmp, num_rbfs, xin[:,2:], fitdat['pry1_vel'], fitdat['pry2_vel'], ctrltype='pv',usingJax=True)



params = jm.initialize_parameters_pv(inputs)

loss_function = jm.create_loss_function_pv(ut.generate_rbf_basis,inputs['num_rbfs'])

grad_loss = jm.compute_loss_gradient(loss_function)



# Set up the optimizer
optimizer, opt_state = jm.setup_optimizer(params, learning_rate=1e-3)

# Number of optimization steps
num_steps = 5000
# Optimization loop
for step in range(num_steps):
    params, opt_state, loss = jm.optimization_step(params, opt_state, optimizer, loss_function, inputs)

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")
        
#transform paramteres to correct domain
Ls_fit=jnp.log(1+jnp.exp(params[1][1:].reshape(2,2)))
L1=Ls_fit[0,:]
L2=Ls_fit[1,:]

weights = params[0]
widths = params[1][0]
w1,w2 = ut.generate_sim_switch(inputs, widths, weights)




resim=sim.controller_sim_pv_post(np.vstack((w1,w2)),xin[:,:2],xin[:,2:],fitdat['pry1_pos'],fitdat['pry1_vel'],fitdat['pry2_pos'],fitdat['pry2_vel'],L1,L2)



diff ev approach

In [ ]:
loss_function = conmod.create_loss_function_pv(conmod.generate_rbf_basis)
grad_loss = conmod.compute_loss_gradient(loss_function)
(L1_opt, L2_opt), best_params, best_loss = conmod.outer_optimization_pv(inputs, conmod.inner_optimization_pv, loss_function, grad_loss,maxiter=30,opttype='local')


loss_function = conmod.create_loss_function_pv(conmod.generate_rbf_basis)
grad_loss = conmod.compute_loss_gradient(loss_function)
(L1_opt, L2_opt), best_params, best_loss = conmod.outer_optimization_pv(inputs, conmod.inner_optimization_pv, loss_function, grad_loss,maxiter=30,opttype='global')


weights = best_params[0:-1][0]
widths = best_params[-1]
w1,w2 = conmod.generate_sim_switch(inputs, widths, weights)

